Hamf taoj node va tao canh cua do thi

In [ ]:

import mysql.connector
from tqdm import tqdm
import json
# Kết nối tới MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="231123",
    database="cv"
)
cur = conn.cursor()
def add_node(node_name, node_type):
    cur.execute("SELECT id FROM nodes WHERE node_name=%s", (node_name,))
    res = cur.fetchone()
    if res:
        return res[0]
    
    cur.execute("SELECT COALESCE(MAX(id), -1) + 1 FROM nodes")
    next_id = cur.fetchone()[0]

    cur.execute("INSERT INTO nodes (id, node_name, node_type) VALUES (%s, %s, %s)", (next_id, node_name, node_type))
    conn.commit()
    return next_id

def add_relation(relation_name):
    cur.execute("SELECT id FROM relations WHERE relation_name=%s", (relation_name,))
    res = cur.fetchone()
    if res:
        return res[0]
    
    cur.execute("SELECT COALESCE(MAX(id), -1) + 1 FROM relations")
    next_id = cur.fetchone()[0]

    cur.execute("INSERT INTO relations (id, relation_name) VALUES (%s, %s)", (next_id, relation_name))
    conn.commit()
    return next_id

def add_edge(head_id, relation_name, tail_id, attributes=None):
    relation_id = add_relation(relation_name)
    cur.execute("""
        SELECT id FROM edges 
        WHERE head_node_id=%s AND relation_id=%s AND tail_node_id=%s
    """, (head_id, relation_id, tail_id))
    if cur.fetchone():
        return
    
    cur.execute("SELECT COALESCE(MAX(id), -1) + 1 FROM edges")
    next_id = cur.fetchone()[0]

    if attributes:
        cur.execute("""
            INSERT INTO edges (id, head_node_id, relation_id, tail_node_id, attributes) 
            VALUES (%s, %s, %s, %s, %s)
        """, (next_id, head_id, relation_id, tail_id, json.dumps(attributes)))
    else:
        cur.execute("""
            INSERT INTO edges (id, head_node_id, relation_id, tail_node_id) 
            VALUES (%s, %s, %s, %s)
        """, (next_id, head_id, relation_id, tail_id))
    conn.commit()


tạo KG trong bảng candidate

In [6]:

import mysql.connector
from tqdm import tqdm

# Kết nối tới MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="231123",
    database="cv"
)
cur = conn.cursor()

cur.execute("select candidate_id, position_title, domain_area, skill from cv.candidate")

rows = cur.fetchall()

for row in tqdm(rows):
    candidate_id = row[0] 
    position_title = row[1]
    domain_area = row[2]
    skill = row[3]

    cand_node_name = f"candidate_{candidate_id}"
    cand_id = add_node(cand_node_name, "candidate")
    # print(cand_id)
    #tao quan he position

    if position_title is not None and position_title.strip() != "":
        pos_node_name = f"position_{position_title.strip().lower().replace(' ', '_')}"
        pos_id = add_node(pos_node_name, "position")
        add_edge(cand_id, "HAS_POSITION_CURRENT", pos_id)
    
    # Tạo quan hệ với domain_area 
    if domain_area is not None and domain_area.strip() != "":
        domains = [d.strip().lower().replace(' ', '_') for d in domain_area.split(',') if d.strip()]
        for domain in domains:
            domain_node_name = f"domain_{domain}"
            domain_id = add_node(domain_node_name, "domain")
            add_edge(cand_id, "WORK_AT_DOMAIN", domain_id)
    

    # Tạo quan hệ với skill nếu tồn tại và không rỗng
    if skill is not None and skill.strip() != "":
        skills = [s.strip().lower().replace(' ', '_') for s in skill.split(',') if s.strip()]
        for sk in skills:
            skill_node_name = f"skill_{sk}"
            skill_id = add_node(skill_node_name, "skill")
            add_edge(cand_id, "HAS_SKILL", skill_id)

cur.close()
conn.close()
    

100%|██████████| 987/987 [04:18<00:00,  3.82it/s]


Tạo KG trong bảng experience

In [10]:
import mysql.connector
from tqdm import tqdm
import json

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="231123",
    database="cv"
)
cur = conn.cursor()

# Đọc bảng experience
cur.execute("SELECT candidate_id, years_experience, job_role, skill, title FROM experience limit 6000, 4000")
rows = cur.fetchall()

# print(rows)

for row in tqdm(rows):
    candidate_id, year_experience, job_role, skill, title = row
    
    if not candidate_id or not job_role:
        continue

    cand_node_name = f"candidate_{candidate_id}"
    # Tìm ID ứng viên (đã có sẵn từ trước)
    cur.execute("SELECT id FROM nodes WHERE node_name=%s", (cand_node_name,))
    cand_row = cur.fetchone()
    if not cand_row:
        continue  # Không tồn tại ứng viên → bỏ qua
    cand_id = cand_row[0]

    # Tạo node công việc
    job_node_name = f"job_{job_role.strip().lower().replace(' ', '_')}"
    job_id = add_node(job_node_name, "job")

    # Tạo quan hệ HAS_EXPERIENCE kèm số năm
    if year_experience and int(year_experience) > 0:
        add_edge(cand_id, "HAS_EXPERIENCE", job_id, attributes={"years": int(year_experience)})
    else:
        add_edge(cand_id, "HAS_EXPERIENCE", job_id)

    # Tạo các kỹ năng yêu cầu bởi job_role
    if skill and skill.strip():
        skills = [s.strip().lower().replace(' ', '_') for s in skill.split(',') if s.strip()]
        for sk in skills:
            skill_node_name = f"skill_{sk}"
            skill_id = add_node(skill_node_name, "skill")
            add_edge(job_id, "REQUIRES_SKILL", skill_id)
    
    # Tạo node title và quan hệ HAS_TITLE
    if title and title.strip():
        title_node_name = f"title_{title.strip().lower().replace(' ', '_')}"
        title_id = add_node(title_node_name, "job_title")
        add_edge(job_id, "HAS_TITLE", title_id)

cur.close()
conn.close()


100%|██████████| 194/194 [00:33<00:00,  5.73it/s]


In [4]:
import mysql.connector
from tqdm import tqdm

# Kết nối đến database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="231123",
    database="cv"
)
cur = conn.cursor(dictionary=True)

# Lấy tất cả edges có relation_id = 3
cur.execute("SELECT head_node_id, tail_node_id FROM edges WHERE relation_id = 3")
edges = cur.fetchall()

# Duyệt từng edge
for edge in tqdm(edges):
    head_id = edge["head_node_id"]
    tail_id = edge["tail_node_id"]

    # Lấy node_name của head (candidate)
    cur.execute("SELECT node_name FROM nodes WHERE id = %s", (head_id,))
    node_candidate = cur.fetchone()
    
    candidate_id = node_candidate["node_name"].replace("candidate_", "")

    # Lấy node_name của tail (job)
    cur.execute("SELECT node_name FROM nodes WHERE id = %s", (tail_id,))
    node_job = cur.fetchone()
    
    job_role = node_job["node_name"].replace("job_", "").replace("_", " ")

    # Cập nhật job_id vào bảng experience
    cur.execute("""
        UPDATE experience
        SET job_id = %s
        WHERE job_role = %s AND candidate_id = %s
    """, (tail_id, job_role, candidate_id))

    # Commit thay đổi sau khi hoàn tất
    conn.commit()


# Đóng kết nối
cur.close()
conn.close()


100%|██████████| 5211/5211 [03:50<00:00, 22.56it/s]
